<a href="https://colab.research.google.com/github/shubham13596/GPT-from-scratch_experiments/blob/main/Gpt_from_scratch_myself.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First building a bigram model

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
#hyperparameters
batch_size = 32 # how many independent sequences processed in parallel
block_size = 8 # max context length for prediction
max_iters = 3000
eval_interval = 300
learning_rate = 1e-02
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200


In [ ]:
torch.manual_seed(1337)

In [ ]:
#wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
  text = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'input.txt'

In [ ]:
chars = sorted(list(set(text)))

In [ ]:
vocab_size = len(chars)

In [ ]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
#train and test splits
data = torch.tensor(encode(text), dtype = torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
#data loading
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+1+block_size] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

In [ ]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out


In [ ]:
#super simple bigram model
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets = None):
    #idx and targets are both tensors of (B,T) tensor of integer
    logits = self.token_embedding_table(idx)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    #idx is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      #get the predictions
      logits, loss = self(idx)
      # focus only on the last time step
      logits = logits[:, -1, :]
      #apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) #(B,C)
      #sample from the distributio
      idx_next = torch.multinomial(probs, num_samples = 1) #B,1
      #append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

In [ ]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

#create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
  if iter%eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  #sample a batch of data
  xb, yb = get_batch("train")

  #evaluate the loss
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

#generate from the model
context = torch.zeros((1,1), dtype = torch.long, device = device)
print(decode(m.generate(context, max_new_tokens = 500)[0].tolist()))

In [ ]:
torch.manual_seed(137)
torch.randint(63,(3,))

tensor([41, 31,  5])

Now building GPT

In [ ]:
#okay what do we need to do
#first define a head through a class
#then define multi head (mke sure there is a linear layer at the last)
# then define a block
# then define a gptlanguage model

#the steps are usual
"""
the steps are usual
1. take the raw data
2. find unqiue vocab to give you vocab_size
3. there is a certain batch_size and n_embd
4. this time we will also have a head_size and number of heads
5. define it later
6. there will be a similar loss function as well.. that nothing changes
7. there will be a similar optimizer as well
8.
"""


'df sdf'

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

#hyperparameters
batch_size = 64 # how many independent sequences processed in parallel
block_size = 256 # max context length for prediction
max_iters = 2500
eval_interval = 500
learning_rate = 3e-04
eval_iters = 200
n_embd = 256
n_head = 6
n_layer = 6
dropout = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
torch.manual_seed(1337)

In [ ]:
with open('input_lalaland.txt', 'r', encoding = 'utf-8') as f:
  text = f.read()

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join(itos[i] for i in l)

In [ ]:
data = torch.tensor(encode(text), dtype = torch.long, device = device)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [ ]:
class Head(nn.Module):
  #defining one head of attention
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias = False)
    self.query = nn.Linear(n_embd, head_size, bias = False)
    self.value = nn.Linear(n_embd, head_size, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self,x):

    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    v = self.value(x)

    wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
    wei = F.softmax(wei, dim = -1)
    wei = self.dropout(wei)
    out = wei @ v
    return out

In [ ]:
class MultiHeadAttention(nn.Module):

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])# understand why nn.ModuleList??
    self.proj = nn.Linear(head_size*num_heads, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim = -1)
    out = self.dropout(self.proj(out))
    return out

In [ ]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):

  def __init__(self, n_embd, n_head):
    # n_embd: embedding dimension, n_head: the number of heads we'd like
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(head_size, n_head)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [ ]:
class GPTLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd) #final layer norm
    self.lm_head = nn.Linear(n_embd, vocab_size)
    self.apply(self._init_weights)

  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
      nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
      if module.bias is not None:
        nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
      nn.init.normal_(module.weight, mean = 0.0, std = 0.02)

  def forward(self, idx, targets = None):
    B, T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device = device))
    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)

    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # crop idx to the last block_size tokens
      idx_cond = idx[:, -block_size:]
      # get the predictions
      logits, loss = self(idx_cond)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim = 1) # check why dim is 1 instead of -1?
    return idx


In [ ]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
  if iter % eval_interval == 0 or iter == max_iters - 1:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch(split="train")
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()


4.820055 M parameters
step 0: train loss 4.5341, val loss 4.5356
step 500: train loss 1.8494, val loss 2.0740
step 1000: train loss 0.8615, val loss 1.8938
step 1500: train loss 0.2509, val loss 2.4000
step 2000: train loss 0.1147, val loss 2.8260
step 2499: train loss 0.0890, val loss 3.1083


In [ ]:
#generate from the model
context = torch.zeros((1,1), dtype = torch.long, device = device)
print(decode(m.generate(context, max_new_tokens = 500)[0].tolist()))


Nor Mia -- and Sebastian his with that -- twas tre the library
reftror an of her Mia fis -- when -MIA
How orkidom rody.
TRACY
I’ve alpos a stree Carlying -- I cut tabo.
Do you be any theare rie?
MIA
(finally in --- his In the corner...
(asmile; a beat)
Well-grees Mia feels. Throse. Gethen, trypect into her players reflection the
their paino f jacois. Thest the fronem iss the
look at its lenot oTV.
LAT. Somets down opps a trackes. A we second. The band finishes away.
10’s the THEAR FAT EITSE]
Mia


In [ ]:
'''

Manual logging:
Experimenet 1
Total params: 10.78Mn
Iterations: 500
Batch size: 64
n_embd = 385
n_layers = 6
batch_size = 64 # how many independent sequences processed in parallel
block_size = 256 # max context length for prediction
max_iters = 500
eval_interval = 500
learning_rate = 3e-04
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

Step 0: 4.8 loss
step 0: train loss 4.2253, val loss 4.2271
step 499: train loss 1.9333, val loss 2.0483

ISIVSul:
Dicive manke tighte  nen't degrainly, Yould thaklt
Now seacke beaght bugut st sof thelme.

CAPULIENET:
Whit notaret: soorce most' buty wou clouad ad
Sprinip teaing frve'sed ist yous.

CAMILLIO:
Terfomet, roveces?

DUpACELIUS:
Thll mongst ay mil; thore, ree, to-botht lod I plee,
And twall day grirdll yourd
FYourst whee grem, arver'd, thy I gras tarkeat,
Lody Mat a he sto crild, mold hay my vomact-oin! He kn. yoif Matuke-gbay

Seast I plind. Tings iteam, scofty, I shom to hire fred,
At pl

Experiment 2:
input = la la land script
batch_size = 64 # how many independent sequences processed in parallel
block_size = 256 # max context length for prediction
max_iters = 2500
eval_interval = 500
learning_rate = 3e-04
eval_iters = 200
n_embd = 256
n_head = 6
n_layer = 6
dropout = 0.2
4.820055 M parameters
step 0: train loss 4.5341, val loss 4.5356
step 500: train loss 1.8494, val loss 2.0740
step 1000: train loss 0.8615, val loss 1.8938
step 1500: train loss 0.2509, val loss 2.4000
step 2000: train loss 0.1147, val loss 2.8260
step 2499: train loss 0.0890, val loss 3.1083

Nor Mia -- and Sebastian his with that -- twas tre the library
reftror an of her Mia fis -- when -MIA
How orkidom rody.
TRACY
I’ve alpos a stree Carlying -- I cut tabo.
Do you be any theare rie?
MIA
(finally in --- his In the corner...
(asmile; a beat)
Well-grees Mia feels. Throse. Gethen, trypect into her players reflection the
their paino f jacois. Thest the fronem iss the
look at its lenot oTV.
LAT. Somets down opps a trackes. A we second. The band finishes away.
10’s the THEAR FAT EITSE]
Mia

'''

SyntaxError: incomplete input (<ipython-input-16-e4615a684824>, line 1)